In [1]:
import pandas as pd
df = pd.read_csv("fake_job_postings.csv")

In [5]:
df.columns

Index(['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent', 'min_salary', 'max_salary'],
      dtype='object')

In [7]:
df.isnull().sum()

job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2696
benefits                7212
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
min_salary             15017
max_salary             15034
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   job_id               17880 non-null  int64  
 1   title                17880 non-null  object 
 2   location             17534 non-null  object 
 3   department           6333 non-null   object 
 4   salary_range         2868 non-null   object 
 5   company_profile      14572 non-null  object 
 6   description          17879 non-null  object 
 7   requirements         15184 non-null  object 
 8   benefits             10668 non-null  object 
 9   telecommuting        17880 non-null  int64  
 10  has_company_logo     17880 non-null  int64  
 11  has_questions        17880 non-null  int64  
 12  employment_type      14409 non-null  object 
 13  required_experience  10830 non-null  object 
 14  required_education   9775 non-null   object 
 15  industry             12977 non-null 

In [9]:
df.describe()

,job_id,telecommuting,has_company_logo,has_questions,fraudulent,min_salary,max_salary
count,17880.000000,17880.000000,17880.000000,17880.000000,17880.000000,2.863000e+03,2.846000e+03
mean,8940.500000,0.042897,0.795302,0.491723,0.048434,5.116296e+05,8.111440e+05
std,5161.655742,0.202631,0.403492,0.499945,0.214688,1.762815e+07,2.703359e+07
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
25%,4470.750000,0.000000,1.000000,0.000000,0.000000,1.800000e+04,2.500000e+04
50%,8940.500000,0.000000,1.000000,0.000000,0.000000,3.500000e+04,5.000000e+04
75%,13410.250000,0.000000,1.000000,1.000000,0.000000,6.000000e+04,9.000000e+04
max,17880.000000,1.000000,1.000000,1.000000,1.000000,8.000000e+08,1.200000e+09
